In [1]:
import pandas as pd
import ast
import numpy as np
from pathlib import Path  
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import geopandas as gpd
import pandas as pd
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Patch
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.colors as mcolors

import os
# os.environ["OMP_NUM_THREADS"] = "2"

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV

from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_percentage_error, mean_squared_error

#import plotly.express as px
import urllib.request
import zipfile
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
from sklearn.decomposition import PCA, FactorAnalysis
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity

from factor_analyzer import FactorAnalyzer

import warnings
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak")


# Load the Excel file
df = pd.read_csv('../../data/raw/msa-municipality.csv')
CDD = pd.read_csv('../../data/raw/cooling_degree_days.csv', skiprows=4, on_bad_lines='skip')
# Create a new dataframe
data = []

# Iterate through each row in the original dataframe
for index, row in df.iterrows():
    msa = row.iloc[0]  # Get the MSA
    counties = ast.literal_eval(row.iloc[1])  # Convert the string representation of the list back to a list
    
    # Append each county with its corresponding MSA to the new data list
    for county in counties:
        data.append([county, msa])

# Create a new dataframe from the data list
msa_county = pd.DataFrame(data, columns=['County', 'MSA'])

# Separate county and state
msa_county [['County Name', 'State']] = msa_county ['County'].str.split(', ', expand=True)
msa_county  = msa_county[['County Name', 'State', 'MSA']]

# Remove the word "County" from the "County Name" column
msa_county['County Name'] = msa_county['County Name'].str.replace('County', '').str.strip()


msa_county

C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,County Name,State,MSA
0,Callahan,TX,"Abilene, TX"
1,Jones,TX,"Abilene, TX"
2,Taylor,TX,"Abilene, TX"
3,Portage,OH,"Akron, OH"
4,Summit,OH,"Akron, OH"
...,...,...,...
1176,Trumbull,OH,"Youngstown-Warren-Boardman, OH-PA"
1177,Mercer,PA,"Youngstown-Warren-Boardman, OH-PA"
1178,Sutter,CA,"Yuba City, CA"
1179,Yuba,CA,"Yuba City, CA"


In [2]:
# Dictionary to map full state names to state abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', ' D.C.': 'DC'
}

# Step 1: Clean the first dataframe by removing 'County' from the 'Name' column and converting state names to abbreviations
CDD['Name'] = CDD['Name'].str.replace(' County', '', regex=False)
CDD['State'] = CDD['State'].apply(lambda x: state_abbreviations.get(x, x))

In [3]:
merged_data = pd.merge(CDD, msa_county, left_on=['Name', 'State'], right_on=['County Name', 'State'], how='left')

In [4]:
# Assuming your DataFrame is named df
merged_data = merged_data.dropna(subset=['MSA'])
merged_data

,ID,Name,State,Value,Anomaly (1901-2000 base period),Rank,1901-2000 Mean,County Name,MSA
0,AL-001,Autauga,AL,2340,249,111,2091,Autauga,"Montgomery, AL"
1,AL-003,Baldwin,AL,2728,287,115,2441,Baldwin,"Daphne-Fairhope-Foley, AL"
3,AL-007,Bibb,AL,1953,-6,54,1959,Bibb,"Birmingham-Hoover, AL"
4,AL-009,Blount,AL,1757,93,81,1664,Blount,"Birmingham-Hoover, AL"
7,AL-015,Calhoun,AL,1798,122,87,1676,Calhoun,"Anniston-Oxford, AL"
...,...,...,...,...,...,...,...,...,...
3082,WI-131,Washington,WI,579,125,108,454,Washington,"Milwaukee-Waukesha, WI"
3083,WI-133,Waukesha,WI,654,116,101,538,Waukesha,"Milwaukee-Waukesha, WI"
3086,WI-139,Winnebago,WI,661,182,115,479,Winnebago,"Oshkosh-Neenah, WI"
3098,WY-021,Laramie,WY,492,252,126,240,Laramie,"Cheyenne, WY"


In [5]:
merged_data['Value'] = pd.to_numeric(merged_data['Value'])

grouped_msa_sector = merged_data.groupby(['MSA'])['Value'].sum().reset_index()
grouped_msa_sector.rename(columns={'Value': 'Cooling Degree Days'}, inplace=True)
grouped_msa_sector

,MSA,Cooling Degree Days
0,"Abilene, TX",7149
1,"Akron, OH",1646
2,"Albany, GA",10187
3,"Albany-Lebanon, OR",163
4,"Albany-Schenectady-Troy, NY",2647
...,...,...
364,"Yakima, WA",254
365,"York-Hanover, PA",1050
366,"Youngstown-Warren-Boardman, OH-PA",1929
367,"Yuba City, CA",3939


In [6]:
#take the average value of the summation of heating degree days
result = merged_data.groupby('MSA')['Value'].sum() / merged_data.groupby('MSA')['County Name'].count()

In [7]:
# Convert the Series to DataFrame
result = result.reset_index()
result.columns = ['MSA', 'Cooling Degree Days']
result

,MSA,Cooling Degree Days
0,"Abilene, TX",2383.00
1,"Akron, OH",823.00
2,"Albany, GA",2546.75
3,"Albany-Lebanon, OR",163.00
4,"Albany-Schenectady-Troy, NY",529.40
...,...,...
364,"Yakima, WA",254.00
365,"York-Hanover, PA",1050.00
366,"Youngstown-Warren-Boardman, OH-PA",643.00
367,"Yuba City, CA",1969.50


In [8]:
result.to_csv('../../data/tidy/cooling-degree-days-average.csv', index=False)
grouped_msa_sector.to_csv('../../data/tidy/cooling-degree-days-sum.csv', index=False)